In [1]:
import os

In [2]:
from unified_planning.shortcuts import *
from unified_planning.io import PDDLReader

In [3]:
os.listdir('pddl/easy')

['opening_presents.pddl',
 'locking_every_window.pddl',
 'picking_up_take-out_food.pddl',
 'opening_packages.pddl',
 'packing_food_for_work.pddl',
 'cleaning_out_drawers.pddl',
 'locking_every_door.pddl',
 'bringing_in_wood.pddl',
 '.ipynb_checkpoints']

In [25]:
reader = PDDLReader()

pddl_dir = 'pddl' 
split_dir = os.path.join(pddl_dir, 'easy')
domain_file = os.path.join(pddl_dir,'domain.pddl')
test_activity_files = [file for file in os.listdir(split_dir) if file!='.ipynb_checkpoints']

for test_activity_file in test_activity_files:
    test_activity_file = os.path.join(split_dir, test_activity_file)
    print(f"\nLooking for plan for {test_activity_file}")
    problem = reader.parse_problem(domain_file, test_activity_file)
    
    # Make a plan for the problem
    with OneshotPlanner(problem_kind=problem.kind) as planner:
        result = planner.solve(problem)
        
        # print(result.log_messages)
        
    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print("Plan found.")
        for action in result.plan.actions:
            print(action)
    else:
        print("No plan found.")


Looking for plan for pddl/easy/cleaning_out_drawers_nextto.pddl
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 14 of `/tmp/ipykernel_3944104/1771105493.py`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

Plan found.
navigate-to(cabinet_1)
open-container(cabinet_1)
grasp(bowl_1)
navigate-to(sink_1)
place-next-to(bowl_1, sink_1)
navigate-to(bowl_2)
grasp(bowl_2)
navigate-to(piece_of_cloth_1)
navigate-to(sink_1)
place-next-to(bowl_2, sink_1)
navigate-to(piece_of_cloth_1)
grasp(piece_of_cloth_1)
navigate-to(sink_1)
place-next-to(piece_of_cloth_1, sink_1)
navigate-to(cabinet_2)
open-container(cabinet_2)
grasp(spoon_1)
navigate-to(sink_1)
place-on(spoon_1, sink_1)
navigate-to(spoon_2)
grasp(spoon_2)
navigate-to(sink_1)
place-on(spoon_2, sink_1)

Looking 

### Exploration of bddl

In [1]:
import os
import bddl

In [2]:
activities = sorted(
        [
            item
            for item in os.listdir(os.path.join(os.path.dirname(bddl.__file__), "activity_definitions"))
            if item != "domain_igibson.bddl"
        ]
    )

assert len(activities)==100, f"Only {len(activities)} activities found."

In [3]:
os.path.join(os.path.dirname(bddl.__file__), "activity_definitions")

'/miniconda/envs/igibson/lib/python3.8/site-packages/bddl/activity_definitions'

In [5]:
activities

['assembling_gift_baskets',
 'bottling_fruit',
 'boxing_books_up_for_storage',
 'bringing_in_wood',
 'brushing_lint_off_clothing',
 'chopping_vegetables',
 'cleaning_a_car',
 'cleaning_barbecue_grill',
 'cleaning_bathrooms',
 'cleaning_bathtub',
 'cleaning_bedroom',
 'cleaning_carpets',
 'cleaning_closet',
 'cleaning_cupboards',
 'cleaning_floors',
 'cleaning_freezer',
 'cleaning_garage',
 'cleaning_high_chair',
 'cleaning_kitchen_cupboard',
 'cleaning_microwave_oven',
 'cleaning_out_drawers',
 'cleaning_oven',
 'cleaning_shoes',
 'cleaning_sneakers',
 'cleaning_stove',
 'cleaning_table_after_clearing',
 'cleaning_the_hot_tub',
 'cleaning_the_pool',
 'cleaning_toilet',
 'cleaning_up_after_a_meal',
 'cleaning_up_refrigerator',
 'cleaning_up_the_kitchen_only',
 'cleaning_windows',
 'clearing_the_table_after_dinner',
 'collect_misplaced_items',
 'collecting_aluminum_cans',
 'defrosting_freezer',
 'filling_a_Christmas_stocking',
 'filling_an_Easter_basket',
 'installing_a_fax_machine',
 'i

In [6]:
domain_file = os.path.join(os.path.dirname(bddl.__file__), "activity_definitions", "domain_igibson.bddl")
with open(domain_file, "r") as f:
    domain = f.read()
    
print(domain)

(define (domain igibson)
    (:requirements :strips :adl)
    (:predicates 
        (inside ?obj1 ?obj2)
        (nextto ?obj1 ?obj2)
        (ontop ?obj1 ?obj2)
        (under ?obj1 ?obj2)
        (broken ?obj1)
        (burnt ?obj1)
        (cooked ?obj1)
        (dusty ?obj1)
        (frozen ?obj1)
        (open ?obj1)
        (perished ?obj1)
        (screwed ?obj1)
        (stained ?obj1)
        (sliced ?obj1)
        (soaked ?obj1)
        (timeset ?obj1)
        (toggled_on ?obj1)
    )
)



In [7]:
predicates_start = domain.find(":predicates")
all_predicates = []
for i, line in enumerate(domain[predicates_start:].split("\n")[1:]):
    if line.strip() == ")":
        break
    predicate = line.strip().split()[0][1:]
    print(f"Predicate {i+1}: {predicate}")
    all_predicates.append(predicate)
    
print(f"Total predicates: {len(all_predicates)}")

Predicate 1: inside
Predicate 2: nextto
Predicate 3: ontop
Predicate 4: under
Predicate 5: broken
Predicate 6: burnt
Predicate 7: cooked
Predicate 8: dusty
Predicate 9: frozen
Predicate 10: open
Predicate 11: perished
Predicate 12: screwed
Predicate 13: stained
Predicate 14: sliced
Predicate 15: soaked
Predicate 16: timeset
Predicate 17: toggled_on
Total predicates: 17


In [8]:
from bddl.parsing import parse_problem, parse_predicates

In [9]:
def parse_predicates_custom(state, all_predicates, found_predicates=None):
    if found_predicates is None:
        found_predicates = {'known': set(), 'unknown': set()}
        
    for item in state:
        # print(item)
        if item in ['and', 'or', 'not', 'forall', 'exists', 'forpairs', 'forn']:
            continue
        if type(item) == list and len(item) == 1:
            continue
        if item[0].startswith("?"): # Ignore variables (type)
            continue
        if type(item[0]) == str and type(item[1]) == str:
            if item[0] in all_predicates:
                found_predicates['known'].add(item[0])
            else:
                found_predicates['unknown'].add(item[0])
        else:
            parse_predicates_custom(item, all_predicates, found_predicates)
            
    return found_predicates

In [10]:
activity = activities[5]
print(activity)

problem, objects, initial_state, goal_state = parse_problem(activity, "0", domain_name="igibson")
parse_predicates_custom(goal_state, all_predicates)

chopping_vegetables


{'known': {'inside', 'sliced'}, 'unknown': set()}

In [11]:
predicates_for_activity = {}

for activity in activities:
    problem, objects, initial_state, goal_state = parse_problem(activity, "0", domain_name="igibson")
    found_predicates = parse_predicates_custom(initial_state, all_predicates)
    found_predicates_for_goal = parse_predicates_custom(goal_state, all_predicates)
    found_predicates['known'] = found_predicates['known'].union(found_predicates_for_goal['known'])
    found_predicates['unknown'] = found_predicates['unknown'].union(found_predicates_for_goal['unknown'])
    predicates_for_activity[activity] = found_predicates

In [12]:
# Print a global list of predicates (known and unknown), count how many activities they appear in

all_predicates = set()
all_predicates_known = set()


for activity, predicates in predicates_for_activity.items():
    all_predicates.update(predicates['known'])
    all_predicates_known.update(predicates['known'])
    all_predicates.update(predicates['unknown'])
    
predicate_count = {predicate: 0 for predicate in all_predicates}
for activity, predicates in predicates_for_activity.items():
    for predicate in predicates['known']:
        predicate_count[predicate] += 1
    for predicate in predicates['unknown']:
        predicate_count[predicate] += 1
        
predicate_count = {k: v for k, v in sorted(predicate_count.items(), key=lambda item: item[1], reverse=True)}

for predicate, count in predicate_count.items():
    print(f"{predicate} ({'known' if predicate in all_predicates_known else 'unknown'}): {count}")

inroom (unknown): 100
onfloor (unknown): 100
ontop (known): 69
inside (known): 67
stained (known): 27
nextto (known): 26
dusty (known): 24
soaked (known): 13
open (known): 7
under (known): 6
toggled_on (known): 6
sliced (known): 5
cooked (known): 3
touching (unknown): 2
frozen (known): 2


In [13]:
predicate_activities = {predicate: [] for predicate in all_predicates}
for activity, predicates in predicates_for_activity.items():
    for predicate in predicates['known']:
        predicate_activities[predicate].append(activity)
    for predicate in predicates['unknown']:
        predicate_activities[predicate].append(activity)
        
for predicate, acts in predicate_activities.items():
    print(f"{predicate}: {acts}")

open: ['bottling_fruit', 'locking_every_door', 'locking_every_window', 'opening_packages', 'opening_presents', 'packing_adult_s_bags', 'preserving_food']
inroom: ['assembling_gift_baskets', 'bottling_fruit', 'boxing_books_up_for_storage', 'bringing_in_wood', 'brushing_lint_off_clothing', 'chopping_vegetables', 'cleaning_a_car', 'cleaning_barbecue_grill', 'cleaning_bathrooms', 'cleaning_bathtub', 'cleaning_bedroom', 'cleaning_carpets', 'cleaning_closet', 'cleaning_cupboards', 'cleaning_floors', 'cleaning_freezer', 'cleaning_garage', 'cleaning_high_chair', 'cleaning_kitchen_cupboard', 'cleaning_microwave_oven', 'cleaning_out_drawers', 'cleaning_oven', 'cleaning_shoes', 'cleaning_sneakers', 'cleaning_stove', 'cleaning_table_after_clearing', 'cleaning_the_hot_tub', 'cleaning_the_pool', 'cleaning_toilet', 'cleaning_up_after_a_meal', 'cleaning_up_refrigerator', 'cleaning_up_the_kitchen_only', 'cleaning_windows', 'clearing_the_table_after_dinner', 'collect_misplaced_items', 'collecting_alumin

In [14]:
from bddl.config import get_definition_filename

In [15]:
test_activity = activities[5]
print(test_activity)

chopping_vegetables
